In [60]:
import pandas as pd
import os
import csv


def prepareDir(dir):
    if not os.path.isdir(dir):
        os.mkdir(dir)
    for file in os.listdir(dir):
        os.remove(dir + file)


def detect_csv_delimiter(file_path):
    """
    Detects the delimiter used in a CSV file.

    Parameters:
    - file_path (str): The path to the CSV file.

    Returns:
    - str: The detected delimiter.
    """
    with open(file_path, 'r', newline='') as file:
        dialect = csv.Sniffer().sniff(file.read(1024))
        return dialect.delimiter


def read_file(file_path):
    """
    Reads a file using the appropriate pandas read function based on the file extension.

    Parameters:
    - file_path (str): The path to the file.

    Returns:
    - pd.DataFrame: The DataFrame containing the data from the file.
    """
    file_extension = file_path.split('.')[-1].lower()

    if file_extension == 'csv':
        return pd.read_csv(file_path, sep=detect_csv_delimiter(file_path))
    elif file_extension in ['xls', 'xlsx']:
        return pd.read_excel(file_path)
    elif file_extension == 'json':
        return pd.read_json(file_path)
    else:
        raise ValueError(f"Unsupported file format: {file_extension}")


INPUT = '../data/raw/'
OUTPUT = '../data/processed/'

DATA_NAME = 'newprices.csv'

DATA_PATH = f'{INPUT}{DATA_NAME}'
data = read_file(DATA_PATH)

In [61]:
NAME_MAPPING = {'Policy_Start_Bonus_Malus_Class' : 'BonusMalus', 'Vehicle_age' : 'CarAge', 'Vehicle_weight_empty' : 'CarWeightMin', 'Number_of_seats' : 'NumberOfSeats', 'Driver_Experience' : 'DriverExperience', 'Vehicle_weight_maximum' : 'CarWeightMax', 'Power_range_in_KW' : 'kw', 'Engine_size' : 'engine_size','DriverAge' : 'driver_age', 'PostalCode' : 'PostalCode', 'CarMake' : 'CarMake', 'Model' : 'CarModel', 'Milage' : 'Mileage'}

INDEX_COL = 'policyNr'

PRICE_ANNOTATIONS = '_newprice'

In [62]:
data

,policyNr,DriverAge,PostalCategory_aegon,PostalCategory_waberer,Number_of_seats,Driver_Experience,Latitude,Longitude,Vehicle_weight_empty,Power_range_in_KW,...,Policy_Start_Bonus_Malus_Class_cut_coefficient,CarMake_Model_cut_coefficient,BasePrice_Ominimo_Group_cut_coefficient,Minimal_Price,Ominimo,kh_newprice,waberer_newprice,aegon_newprice,si_newprice,allianz_newprice
0,81002059798,63,4,7,5,38,46.3667,17.8000,1070,55,...,1.286456,1.1,1.100,24328,59816.419175,71644,82740,60180,80779,28200
1,81025491531,27,2,4,5,8,47.6177,19.1887,1080,57,...,1.286456,1.1,1.100,64486,114339.214131,76856,120528,119844,166246,66960
2,81025500513,52,7,10,5,33,46.8463,16.5548,1310,77,...,1.286456,1.1,1.100,21487,59353.187247,56404,47280,49524,64152,34080
3,81021294855,48,2,4,4,30,47.4984,19.0404,1276,70,...,1.286456,1.1,1.000,32732,76352.368270,113016,92796,72408,87227,59520
4,81021337324,28,7,10,5,6,47.1295,16.8069,1280,88,...,1.286456,1.1,1.100,39983,51204.469071,41220,55272,49716,75038,41520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294291,81025258005,47,5,9,5,24,47.5905,17.9186,1054,60,...,1.350456,1.0,1.000,169094,169094.000000,389064,174324,178212,289736,203160
294292,81036181931,42,6,7,5,20,47.9554,21.7167,1765,100,...,1.350456,1.0,1.000,180886,180886.000000,527484,228132,230412,283179,186480
294293,81027121532,0,2,6,5,0,47.3545,19.0982,1581,125,...,1.350456,1.0,1.075,669820,669820.000000,1186096,900300,1465368,1260323,801840
294294,81026476317,46,6,8,5,2,47.6092,19.6028,1079,50,...,1.350456,0.9,1.000,152379,152379.000000,342292,209280,157092,181018,278640


In [63]:
price_columns = data.filter(regex = PRICE_ANNOTATIONS, axis = 1).columns.tolist()
price_columns_mapping = {x : x.replace(PRICE_ANNOTATIONS, '_price') for x in price_columns}

NAME_MAPPING.update(price_columns_mapping)

In [64]:
data = data.rename(NAME_MAPPING, axis = 1)
data = data.set_index(INDEX_COL)

In [65]:
data = data[list(NAME_MAPPING.values())]

In [70]:
data.to_csv(f'{OUTPUT}{DATA_NAME.split(".")[0]}_processed.csv')

In [71]:
file = open(f'{OUTPUT}{DATA_NAME.split(".")[0]}' + '_features.txt','w')
feature_cols = [col for col in data.columns if '_price' not in col]
for feature in feature_cols:
    file.write(feature+ "," + str(data[feature].dtype) + "\n")
file.close()